# 데이터 로드 및 전처리

In [ ]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,mean_absolute_percentage_error
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib

#한글꺠짐 방지
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [2]:
data=pd.read_csv("./data/Fault_data.csv",encoding="EUC-KR")
data.drop(["자재구분","END_DATE_TIME"],axis=1,inplace=True)
data.dropna(axis=1,inplace=True)

#X,Y 분할
Y=data["Y"].copy()
X=data.drop("Y",axis=1)
X.head(3)

,CHAMBER,TAE.STDDEV0,TAE.MEAN0,TAE.RANGE0,PC.STDDEV0,PC.MEAN0,PC.RANGE0,CROT.STDDEV0,CROT.MEAN0,CROT.RANGE0,...,PC.RANGE12,TAE.STDDEV9,TAE.MEAN9,TAE.RANGE9,CROT.STDDEV9,CROT.MEAN9,CROT.RANGE9,PC.STDDEV9,PC.MEAN9,PC.RANGE9
0,4,0.044554,38.083127,0.254154,0.124753,0.044329,0.439883,0,35,0,...,17.350926,0.032832,35.011650,0.127077,0,35,0,0.405343,45.124841,3.079182
1,2,0.034801,38.052395,0.063538,0.177575,0.342131,0.439883,0,27,0,...,1.466280,0.020858,35.550245,0.063540,0,27,0,1.923012,45.350277,15.004887
2,3,0.030089,38.046407,0.127080,0.137559,0.063983,0.488759,0,15,0,...,13.098728,0.021635,36.013373,0.063538,0,15,0,0.161640,45.071393,0.977516


In [3]:
data

,Y,CHAMBER,TAE.STDDEV0,TAE.MEAN0,TAE.RANGE0,PC.STDDEV0,PC.MEAN0,PC.RANGE0,CROT.STDDEV0,CROT.MEAN0,...,PC.RANGE12,TAE.STDDEV9,TAE.MEAN9,TAE.RANGE9,CROT.STDDEV9,CROT.MEAN9,CROT.RANGE9,PC.STDDEV9,PC.MEAN9,PC.RANGE9
0,69,4,0.044554,38.083127,0.254154,0.124753,0.044329,0.439883,0,35,...,17.350926,0.032832,35.011650,0.127077,0,35,0,0.405343,45.124841,3.079182
1,28,2,0.034801,38.052395,0.063538,0.177575,0.342131,0.439883,0,27,...,1.466280,0.020858,35.550245,0.063540,0,27,0,1.923012,45.350277,15.004887
2,13,3,0.030089,38.046407,0.127080,0.137559,0.063983,0.488759,0,15,...,13.098728,0.021635,36.013373,0.063538,0,15,0,0.161640,45.071393,0.977516
3,101,4,0.047904,38.090810,0.190617,1.441465,0.503421,6.549365,0,35,...,17.790812,0.025405,35.020727,0.127077,0,35,0,0.359273,45.110758,2.688171
4,319,3,0.036058,38.023829,0.190617,0.116419,0.045446,0.391007,0,58,...,3.714566,0.076083,33.967311,0.317697,0,58,0,0.447676,45.147192,3.225808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,27,3,0.027780,38.064804,0.063536,0.306206,0.342131,1.124145,0,65,...,10.117297,0.025479,36.533647,0.127079,0,65,0,0.325076,45.069147,3.176930
135,23,3,0.033961,38.077084,0.063536,0.163848,0.502723,0.488759,0,64,...,11.290322,0.033529,36.548598,0.127079,0,64,0,0.668920,45.131644,5.180838
136,56,3,0.031002,38.068007,0.063536,0.177190,0.481776,0.537634,0,64,...,3.567934,0.026805,36.535092,0.127079,0,64,0,0.613599,44.972303,4.985336
137,78,2,0.031002,38.068007,0.063536,0.170316,0.349113,0.439883,0,22,...,2.492667,0.038289,36.022728,0.127080,0,22,0,1.382720,45.285845,10.801564


# 평가 지표 함수

In [4]:
# 평가 지표 출력 함수
def print_all_reg(Y_test,pred):
    print("model의 MSE 값은 {:.3f}".format(mean_squared_error(Y_test,pred)))
    print("model의 MAE 값은 {:.3f}".format(mean_absolute_error(Y_test, pred)))
    print("model의 MAPE 값은 {:.3f}".format(mean_absolute_percentage_error(Y_test, pred)))
    print("model의 r2 값은 {:.3f}".format(r2_score(Y_test, pred)))

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=5,shuffle =True)

# 모델 학습 및 예측

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import math
import time
import datetime


def evaluate_regression_models(X, y, model_name):
    # 데이터 생성    
    # 모델 학습
    if model_name == "GBM":
        # Gradient Boosting 모델
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        model = GradientBoostingRegressor(n_estimators=500,validation_fraction = 0.2, n_iter_no_change = 10)
        model.fit(X_train, y_train)
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(y_train,preds)

    elif model_name == "XGB":        
        # XGBoost 모델
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        eval_set = [(X_eval, Y_eval)]
        model = xgb.XGBRegressor(n_estimators=500,learning_rate = 0.001,
                        min_child_weight = 2,
                        max_depth = 10)
        model.fit(X_train, y_train,eval_set = eval_set,early_stopping_rounds = 10)
        
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(y_train,preds)
    
    elif model_name == "Light":     
        params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

        # LightGBM 모델
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        train_ds = lgb.Dataset(X_train, label = y_train) 
        test_ds = lgb.Dataset(X_eval, label = Y_eval) 
        model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=10)

#         model = lgb.LGBMRegressor(colsample_bytree = 0.7,max_depth = 10,min_child_weight = 2,n_estimators=500,learning_rate = 0.1)
#         model.fit(X_train, y_train,eval_set = eval_set,early_stopping_rounds = 40,eval_metric='mae')
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(y_train,preds)

    elif model_name == "Cat":     
        # CatBoost 모델
        X_train, X_eval, y_train, Y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
        eval_set = [(X_eval, Y_eval)]
        model = cb.CatBoostRegressor(max_depth = 10,n_estimators=1000,learning_rate = 0.02,logging_level='Silent')
        model.fit(X_train, y_train,eval_set = eval_set,early_stopping_rounds = 10)
        print("학습 데이터 성능")
        preds = model.predict(X_train)
        print_all_reg(y_train,preds)

    

    return model


# GBM 평가

In [7]:
print("일반 GBM")
print("테스트 데이터 성능")
start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"GBM")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(Y_test,preds)

일반 GBM
테스트 데이터 성능
학습 데이터 성능
model의 MSE 값은 530.703
model의 MAE 값은 17.991
model의 MAPE 값은 0.494
model의 r2 값은 0.858
Time: 0.0169sec
model의 MSE 값은 2556.254
model의 MAE 값은 39.654
model의 MAPE 값은 2.261
model의 r2 값은 -0.190


# xgboost 평가

In [8]:
# GBM 평가start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"XGB")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(Y_test,preds)

[0]	validation_0-rmse:109.87126
[1]	validation_0-rmse:109.82984
[2]	validation_0-rmse:109.78848
[3]	validation_0-rmse:109.74716
[4]	validation_0-rmse:109.70591
[5]	validation_0-rmse:109.66471
[6]	validation_0-rmse:109.62357
[7]	validation_0-rmse:109.58248
[8]	validation_0-rmse:109.54145
[9]	validation_0-rmse:109.50047
[10]	validation_0-rmse:109.45955
[11]	validation_0-rmse:109.41941
[12]	validation_0-rmse:109.37860
[13]	validation_0-rmse:109.33856
[14]	validation_0-rmse:109.29858
[15]	validation_0-rmse:109.25793
[16]	validation_0-rmse:109.21806
[17]	validation_0-rmse:109.17823
[18]	validation_0-rmse:109.13774
[19]	validation_0-rmse:109.09705
[20]	validation_0-rmse:109.05641
[21]	validation_0-rmse:109.01582
[22]	validation_0-rmse:108.97528
[23]	validation_0-rmse:108.93481
[24]	validation_0-rmse:108.89439
[25]	validation_0-rmse:108.85403
[26]	validation_0-rmse:108.81371
[27]	validation_0-rmse:108.77345
[28]	validation_0-rmse:108.73325
[29]	validation_0-rmse:108.69311
[30]	validation_0-rm

C:\Users\hyeonggu\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[93]	validation_0-rmse:106.23097
[94]	validation_0-rmse:106.19114
[95]	validation_0-rmse:106.15437
[96]	validation_0-rmse:106.11464
[97]	validation_0-rmse:106.07809
[98]	validation_0-rmse:106.03849
[99]	validation_0-rmse:106.00204
[100]	validation_0-rmse:105.96254
[101]	validation_0-rmse:105.92609
[102]	validation_0-rmse:105.88670
[103]	validation_0-rmse:105.85043
[104]	validation_0-rmse:105.81120
[105]	validation_0-rmse:105.77502
[106]	validation_0-rmse:105.73591
[107]	validation_0-rmse:105.69983
[108]	validation_0-rmse:105.66072
[109]	validation_0-rmse:105.62474
[110]	validation_0-rmse:105.58586
[111]	validation_0-rmse:105.54991
[112]	validation_0-rmse:105.51102
[113]	validation_0-rmse:105.47518
[114]	validation_0-rmse:105.43640
[115]	validation_0-rmse:105.40066
[116]	validation_0-rmse:105.36200
[117]	validation_0-rmse:105.32635
[118]	validation_0-rmse:105.28781
[119]	validation_0-rmse:105.25238
[120]	validation_0-rmse:105.21394
[121]	validation_0-rmse:105.17850
[122]	validation_0-rm

[338]	validation_0-rmse:96.57499
[339]	validation_0-rmse:96.54519
[340]	validation_0-rmse:96.49821
[341]	validation_0-rmse:96.44799
[342]	validation_0-rmse:96.41839
[343]	validation_0-rmse:96.38876
[344]	validation_0-rmse:96.36423
[345]	validation_0-rmse:96.33599
[346]	validation_0-rmse:96.31131
[347]	validation_0-rmse:96.28306
[348]	validation_0-rmse:96.25867
[349]	validation_0-rmse:96.23059
[350]	validation_0-rmse:96.20603
[351]	validation_0-rmse:96.17826
[352]	validation_0-rmse:96.12837
[353]	validation_0-rmse:96.10390
[354]	validation_0-rmse:96.07592
[355]	validation_0-rmse:96.05176
[356]	validation_0-rmse:96.02394
[357]	validation_0-rmse:95.99962
[358]	validation_0-rmse:95.97211
[359]	validation_0-rmse:95.94785
[360]	validation_0-rmse:95.92009
[361]	validation_0-rmse:95.89589
[362]	validation_0-rmse:95.87205
[363]	validation_0-rmse:95.82272
[364]	validation_0-rmse:95.79519
[365]	validation_0-rmse:95.76785
[366]	validation_0-rmse:95.74382
[367]	validation_0-rmse:95.71632
[368]	vali

# LightGBM 평가

In [9]:
start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"Light")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(Y_test,preds)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000248 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3023
[LightGBM] [Info] Number of data points in the train set: 100, number of used features: 104
[LightGBM] [Info] Start training from score 67.400000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

C:\Users\hyeonggu\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hyeonggu\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


# Catboost 평가

In [10]:
start_time = time.time()
model= evaluate_regression_models(X_train, Y_train,"Cat")
print("Time: {:.4f}sec".format((time.time() - start_time)))
preds = model.predict(X_test)
print_all_reg(Y_test,preds)

학습 데이터 성능
model의 MSE 값은 97.937
model의 MAE 값은 7.202
model의 MAPE 값은 0.181
model의 r2 값은 0.974
Time: 8.6713sec
model의 MSE 값은 1229.642
model의 MAE 값은 28.507
model의 MAPE 값은 1.592
model의 r2 값은 0.428
